In [30]:
import pyspark.sql.types 

from pyspark import SparkConf, SparkContext

from pyspark.ml import Pipeline

from pyspark.sql.functions import udf,col

from pyspark.sql import SQLContext

from pyspark.ml.regression import DecisionTreeRegressor

from pyspark.ml.feature import VectorIndexer,VectorAssembler

from pyspark.ml.evaluation import RegressionEvaluator

In [31]:
global Path    

Path="file:/home/hadoop/"

In [32]:
def CreateSparkContext():

    def SetLogger( sc ):

        logger = sc._jvm.org.apache.log4j

        logger.LogManager.getLogger("org"). setLevel( logger.Level.ERROR )

        logger.LogManager.getLogger("akka").setLevel( logger.Level.ERROR )

        logger.LogManager.getRootLogger().setLevel(logger.Level.ERROR)

    #sparkConf = SparkConf().setAppName("RunDecisionTreeBinary").set("spark.ui.showConsoleProgress", "false") 

    #sc = SparkContext(conf = sparkConf)

    SetLogger(sc)

    return (sc)

In [33]:
sc=CreateSparkContext()

In [34]:
sqlContext = SQLContext(sc)

In [49]:
train_row_df = sqlContext.read.format("csv").option("header", "true").load(Path+"dataset/shimen_repository.csv")

In [50]:
train_row_df.columns

['region',
 'perticipation',
 'input',
 'output',
 'water_level_difference',
 'effective_store',
 'store_amount',
 'StnPres',
 'Temperature',
 'RH',
 'water_level']

In [37]:
type(train_row_df)

pyspark.sql.dataframe.DataFrame

In [8]:
test_row_df = sqlContext.read.format("csv").option("header", "true").load(Path+"dataset/phei_repository.csv")

In [29]:
test_row_df.show()

+-------+-------------+------+------+----------------------+---------------+------------+-----------+
| region|perticipation| input|output|water_level_difference|effective_store|store_amount|water_level|
+-------+-------------+------+------+----------------------+---------------+------------+-----------+
|33550.5|          7.2|  58.4| 151.1|                 -0.12|        22479.4|        0.67|     157.03|
|33550.5|          0.6| 58.87|151.27|                 -0.11|       22379.26|       0.667|      156.9|
|33550.5|           14| 82.25|151.45|                 -0.12|        22317.8|      0.6652|     156.82|
|33550.5|          0.7| 74.46|151.16|                 -0.07|       22233.47|      0.6627|     156.71|
|33550.5|          0.1| 58.36|180.66|                 -0.11|       22111.14|       0.659|     156.55|
|33550.5|          0.1| 49.12|193.92|                 -0.16|       21974.03|       0.655|     156.37|
|33550.5|            0| 43.65|210.45|                 -0.19|       21822.32|      

In [58]:
print("read data")

new_train_row_df=train_row_df.drop("region").drop("effective_store").drop("store_amount")

train_df= new_train_row_df.select([ col(column).cast("double").alias(column) for column in new_train_row_df.columns])

new_test_row_df=test_row_df.drop("region").drop("effective_store").drop("store_amount")

test_df= new_test_row_df.select([ col(column).cast("double").alias(column) for column in new_test_row_df.columns])

train_df.cache()

test_df.cache()

read data


DataFrame[perticipation: double, input: double, output: double, water_level_difference: double, water_level: double]

In [59]:
assemblerInputs = train_df.columns[:-1]

print("setup pipeline")

assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="tmp_features")

indexer = VectorIndexer(inputCol="tmp_features", outputCol="features", maxCategories=24)

dt = DecisionTreeRegressor(labelCol="water_level",featuresCol= "features",maxDepth=10, maxBins=100,impurity="variance")

dt_pipeline = Pipeline(stages=[assembler,indexer ,dt])

setup pipeline


In [60]:
print("train model")
dt_pipelineModel = dt_pipeline.fit(train_df)

train model


Py4JJavaError: An error occurred while calling o789.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 45.0 failed 1 times, most recent failure: Lost task 0.0 in stage 45.0 (TID 45, localhost, executor driver): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$3: (struct<perticipation:double,input:double,output:double,water_level_difference:double,StnPres:double,Temperature:double,RH:double>) => vector)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$10$$anon$1.hasNext(WholeStageCodegenExec.scala:614)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:408)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:408)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator

In [12]:
print("predict")

predicted=dt_pipelineModel.transform(test_df).select("water_level","prediction").show(10)

print(predicted)

predict
+-----------+----------+
|water_level|prediction|
+-----------+----------+
|     157.03|    239.19|
|      156.9|    239.19|
|     156.82|    239.19|
|     156.71|    244.53|
|     156.55|    239.19|
|     156.37|    239.19|
|     156.17|    240.81|
|     155.95|    239.73|
|     155.73|    239.73|
|     155.51|    239.73|
+-----------+----------+
only showing top 10 rows

None


In [14]:
print("eval model")

evaluator = RegressionEvaluator(labelCol='water_level',predictionCol='prediction',metricName="rmse")

predicted_df=dt_pipelineModel.transform(test_df)

rmse = evaluator.evaluate(predicted_df)

print(rmse)

eval model
82.2906636827
